In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from tqdm import tqdm
import json

# Your session cookie (_t value only, not the full cookie header)
DISCOURSE_COOKIE = "LqY08BQ2M3oFnhYj3l3PXW7liWUmgVvkLvfdAI9aLdATkMVu6RX0R3smIdnNW55kXnW%2FihK3XvOIC0K5G53blgOE0s9Udec4eW0%2FAq1FHJURLuM7wFfPcoIdv4LY6VFmALBmpE4MhRXU7WQDp%2BNmKS0mgNUENPz%2BWPILs0K1oYK91AhXNJzuJyUQiY%2BZe1kCKjPkgmgRaYa%2BrcYZ7yJbFE%2BcT%2BnmF%2BgRc9agvHn%2B0Fp4CHqYrMHDlzEqHQat2cBrZhK0XgAdS0ROyQnsCooZN6gQn44i1Rt7H26X0xWoMQTgg%2BJvve6rX3qGwToS9B2D0GRONqevQ%2BA%3D--sZ1bQwgq%2B9ZP4%2F2J--UmFPNR9KyIhX3uX1heAiMA%3D%3D"

session = requests.Session()
session.cookies.set("_t", DISCOURSE_COOKIE, domain="discourse.onlinedegree.iitm.ac.in")
session.headers.update({"User-Agent": "Mozilla/5.0"})

BASE_URL = "https://discourse.onlinedegree.iitm.ac.in"

def get_topic_ids(category_slug="courses/tds-kb", category_id=34):
    topics = []
    for page in tqdm(range(0, 20)):  # Adjust if you want more pages
        url = f"{BASE_URL}/c/{category_slug}/{category_id}.json?page={page}"
        r = session.get(url)
        if r.status_code != 200:
            break
        data = r.json()
        new_topics = data["topic_list"]["topics"]
        if not new_topics:
            break
        topics.extend(new_topics)
    return topics

def get_posts_in_topic(topic_id):
    r = session.get(f"{BASE_URL}/t/{topic_id}.json")
    if r.status_code != 200:
        return []
    data = r.json()
    return [
        {
            "username": post["username"],
            "created_at": post["created_at"],
            "content": BeautifulSoup(post["cooked"], "html.parser").get_text(),
            "post_url": f"{BASE_URL}/t/{topic_id}/{post['post_number']}"
        }
        for post in data["post_stream"]["posts"]
    ]

all_posts = []
topics = get_topic_ids()

for topic in tqdm(topics):
    # Parse created_at as timezone-aware datetime (UTC)
    created_at = datetime.fromisoformat(topic["created_at"].replace("Z", "+00:00"))
    # Compare with a timezone-aware datetime for Jan 1, 2025 UTC
    if created_at >= datetime(2025, 1, 1, tzinfo=timezone.utc):
        posts = get_posts_in_topic(topic["id"])
        all_posts.extend(posts)

# Save the scraped posts into a JSON file
with open("tds_discourse_posts.json", "w", encoding="utf-8") as f:
    json.dump(all_posts, f, indent=2, ensure_ascii=False)

print(f"Scraped {len(all_posts)} posts.")


100%|█████████████████████████████████████████| 600/600 [02:17<00:00,  4.35it/s]

Scraped 1107 posts.
